# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords

from nltk.sentiment import SentimentAnalyzer

NUM_FEATURES = 5000
TEST_SET_SIZE = 20000

In [4]:
raw_data = pd.read_csv('Sentiment140.csv', encoding="ISO-8859-1")
data = raw_data.sample(TEST_SET_SIZE)
data.head()

,target,ids,date,flag,user,text
1105003,4,1971232408,Sat May 30 06:34:49 PDT 2009,NO_QUERY,fabeku,@Tammy2luvs Tasting poisonous flowers isn't us...
1594980,4,2192270573,Tue Jun 16 06:46:49 PDT 2009,NO_QUERY,raaahbin,@sbleby I'm afraid nephews are not so much an ...
1104586,4,1971180894,Sat May 30 06:26:32 PDT 2009,NO_QUERY,ElwoodJBlues,"Back to the Future on Encore, Bttf2 on Starz. ..."
121628,0,1833552520,Sun May 17 23:39:31 PDT 2009,NO_QUERY,cc_monkey,for once wishes it wasn't miserable weather - ...
483197,0,2180291785,Mon Jun 15 10:16:12 PDT 2009,NO_QUERY,mikki106,Gotta go back to the phone


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# your code here
#cleaning unwanted words
import re

def remove_by_regex(tweet, regexp):
        return re.sub(regexp, '', tweet)

def remove_url(tweet):
    return remove_by_regex(tweet, re.compile(r"http.?://[^\s]+[\s]?"))

def remove_special_char(tweet):
    return re.sub(r"[^a-zA-Z0-9 ]", " ", tweet)#add space placeholder

def remove_numbers(tweet):
    return remove_by_regex(tweet, re.compile(r"[1234567890]"))

def clean_up(tweet):
    tweet = remove_numbers(tweet)
    tweet = remove_url(tweet)
    tweet = remove_special_char(tweet)
    return tweet.lower().strip()
data.text = data.text.apply(lambda tweet: clean_up(tweet))

data.head()

,target,ids,date,flag,user,text
1105003,4,1971232408,Sat May 30 06:34:49 PDT 2009,NO_QUERY,fabeku,tammyluvs tasting poisonous flowers isn t usua...
1594980,4,2192270573,Tue Jun 16 06:46:49 PDT 2009,NO_QUERY,raaahbin,sbleby i m afraid nephews are not so much an o...
1104586,4,1971180894,Sat May 30 06:26:32 PDT 2009,NO_QUERY,ElwoodJBlues,back to the future on encore bttf on starz b...
121628,0,1833552520,Sun May 17 23:39:31 PDT 2009,NO_QUERY,cc_monkey,for once wishes it wasn t miserable weather ...
483197,0,2180291785,Mon Jun 15 10:16:12 PDT 2009,NO_QUERY,mikki106,gotta go back to the phone


In [6]:
#Tokenize
from nltk.tokenize import word_tokenize

data['text_processed'] = data.text.apply(lambda x: word_tokenize(x))

data.head()

,target,ids,date,flag,user,text,text_processed
1105003,4,1971232408,Sat May 30 06:34:49 PDT 2009,NO_QUERY,fabeku,tammyluvs tasting poisonous flowers isn t usua...,"[tammyluvs, tasting, poisonous, flowers, isn, ..."
1594980,4,2192270573,Tue Jun 16 06:46:49 PDT 2009,NO_QUERY,raaahbin,sbleby i m afraid nephews are not so much an o...,"[sbleby, i, m, afraid, nephews, are, not, so, ..."
1104586,4,1971180894,Sat May 30 06:26:32 PDT 2009,NO_QUERY,ElwoodJBlues,back to the future on encore bttf on starz b...,"[back, to, the, future, on, encore, bttf, on, ..."
121628,0,1833552520,Sun May 17 23:39:31 PDT 2009,NO_QUERY,cc_monkey,for once wishes it wasn t miserable weather ...,"[for, once, wishes, it, wasn, t, miserable, we..."
483197,0,2180291785,Mon Jun 15 10:16:12 PDT 2009,NO_QUERY,mikki106,gotta go back to the phone,"[got, ta, go, back, to, the, phone]"


In [8]:
#Stemming and lemmatization
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(tweet):
    return list(map(lambda s: lemmatizer.lemmatize(stemmer.stem(s)), tweet))
data.text_processed = data.text_processed.apply(lambda s: stem_and_lemmatize(s))
data.head()

,target,ids,date,flag,user,text,text_processed
1105003,4,1971232408,Sat May 30 06:34:49 PDT 2009,NO_QUERY,fabeku,tammyluvs tasting poisonous flowers isn t usua...,"[tammyluv, tast, poison, flower, isn, t, usual..."
1594980,4,2192270573,Tue Jun 16 06:46:49 PDT 2009,NO_QUERY,raaahbin,sbleby i m afraid nephews are not so much an o...,"[sblebi, i, m, afraid, nephew, are, not, so, m..."
1104586,4,1971180894,Sat May 30 06:26:32 PDT 2009,NO_QUERY,ElwoodJBlues,back to the future on encore bttf on starz b...,"[back, to, the, futur, on, encor, bttf, on, st..."
121628,0,1833552520,Sun May 17 23:39:31 PDT 2009,NO_QUERY,cc_monkey,for once wishes it wasn t miserable weather ...,"[for, onc, wish, it, wasn, t, miser, weather, ..."
483197,0,2180291785,Mon Jun 15 10:16:12 PDT 2009,NO_QUERY,mikki106,gotta go back to the phone,"[got, ta, go, back, to, the, phone]"


In [10]:
#removing stopwords
stopwords_list = stopwords.words("english")
def remove_stopwords(tweet):
    return [x for x in tweet if x not in stopwords_list]

data.text_processed = data.text_processed.apply(lambda s: remove_stopwords(s))

data.head()

,target,ids,date,flag,user,text,text_processed
1105003,4,1971232408,Sat May 30 06:34:49 PDT 2009,NO_QUERY,fabeku,tammyluvs tasting poisonous flowers isn t usua...,"[tammyluv, tast, poison, flower, usual, great,..."
1594980,4,2192270573,Tue Jun 16 06:46:49 PDT 2009,NO_QUERY,raaahbin,sbleby i m afraid nephews are not so much an o...,"[sblebi, afraid, nephew, much, option, stepdau..."
1104586,4,1971180894,Sat May 30 06:26:32 PDT 2009,NO_QUERY,ElwoodJBlues,back to the future on encore bttf on starz b...,"[back, futur, encor, bttf, starz, film, start,..."
121628,0,1833552520,Sun May 17 23:39:31 PDT 2009,NO_QUERY,cc_monkey,for once wishes it wasn t miserable weather ...,"[onc, wish, miser, weather, peopl, arriv, beau..."
483197,0,2180291785,Mon Jun 15 10:16:12 PDT 2009,NO_QUERY,mikki106,gotta go back to the phone,"[got, ta, go, back, phone]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [16]:
# your code here
all_words = []

for index, value in data.text_processed.iteritems():
    if value not in all_words:
        all_words += value

top_features = list(nltk.FreqDist(all_words).keys())[:NUM_FEATURES]
top_features = top_features[:5000]
top_features

['tammyluv',
 'tast',
 'poison',
 'flower',
 'usual',
 'great',
 'idea',
 'glad',
 'quot',
 'go',
 'sblebi',
 'afraid',
 'nephew',
 'much',
 'option',
 'stepdaught',
 'cool',
 'back',
 'futur',
 'encor',
 'bttf',
 'starz',
 'film',
 'start',
 'time',
 'space',
 'continuum',
 'paradox',
 'live',
 'room',
 'onc',
 'wish',
 'miser',
 'weather',
 'peopl',
 'arriv',
 'beauti',
 'citi',
 'got',
 'ta',
 'phone',
 'ok',
 'send',
 'messag',
 'ask',
 'set',
 'blog',
 'work',
 'link',
 'develop',
 'websit',
 'take',
 'tip',
 'natazzz',
 'patient',
 'would',
 'love',
 'home',
 'babi',
 'bear',
 'arm',
 'right',
 'meow',
 'bodi',
 'hurti',
 'kendallmascott',
 'see',
 'new',
 'hey',
 'show',
 'jay',
 'leno',
 'brilliant',
 'lot',
 'laugh',
 'whoa',
 'know',
 'wa',
 'come',
 'argh',
 'want',
 'sped',
 'money',
 'leav',
 'waterpark',
 'wit',
 'son',
 'good',
 'til',
 'spit',
 'close',
 'pool',
 'lol',
 'instal',
 'window',
 'vista',
 'updat',
 'kati',
 'clark',
 'brain',
 'hurt',
 'way',
 'mani',
 'dr

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [17]:
# your code here
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (w in words)
    return features

In [20]:
featuresets = []
for index, row in data.iterrows():
    featuresets.append((build_features(row['text_processed']), row['target']==4))

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [21]:
# your code here
training, test = train_test_split(featuresets, test_size=0.2)
classifier = nltk.NaiveBayesClassifier.train(training)
classifier.show_most_informative_features()

Most Informative Features
                     sad = True            False : True   =     22.1 : 1.0
                     vip = True             True : False  =     14.6 : 1.0
                   smile = True             True : False  =     13.5 : 1.0
               congratul = True             True : False  =     12.7 : 1.0
                     gut = True            False : True   =     12.7 : 1.0
                    sick = True            False : True   =     11.9 : 1.0
                    hehe = True             True : False  =     11.3 : 1.0
                    poor = True            False : True   =     10.6 : 1.0
                     www = True             True : False  =     10.1 : 1.0
                 horribl = True            False : True   =     10.1 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [22]:
# your code here
print("Accuracy percent:",(nltk.classify.accuracy(classifier, test))*100.00)

Accuracy percent: 72.675


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here